# Data Pre Processing of T20 Cricket World Cup 2022

* Importing required Libraries such as Pandas, JSON, etc.
* Importing ***t20_wc2022_match_results*** json file and
converting it to Data Frame.

In [1]:
import pandas as pd
import json

with open("/t20_wc2022_match_results.json") as f:
  data = json.load(f)
df_match = pd.DataFrame(data[0]["matchSummary"])
df_match.rename({'scorecard':'match_id'},axis=1,inplace=True)
df_match
df_match.to_csv("dim_match_summary.csv", index = False)

Creating a dictionary in order to join **df_match** with **df_batting** and **df_bowling** DataFrames

In [ ]:
match_id_dict = {}

for index,row in df_match.iterrows():
  key1 = row["team1"] + " Vs " + row["team2"]
  key2 = row["team2"] + " Vs " + row["team1"]

  match_id_dict[key1] = row["match_id"]
  match_id_dict[key2] = row["match_id"]

match_id_dict

Importing **t20_wc2022_batting_summary** json file and also performing **data transformation** and **data cleaning**.

In [4]:
with open("/t20_wc2022_batting_summary.json") as f:
  data = json.load(f)
  records = []
  for rec in data:
    records.extend(rec["battingSummary"])
df_batting = pd.DataFrame(records)
df_batting["out/not_out"] = df_batting["dismissal"].apply(lambda x: "out" if len(x)>0 else "not_out")
df_batting.drop("dismissal",axis='columns',inplace=True)
df_batting["batsmanName"] = df_batting["batsmanName"].apply(lambda x: x.replace("†",""))
df_batting["match_id"] = df_batting["match"].map(match_id_dict)
df_batting.head()
df_batting.to_csv("fact_batting_summary.csv", index = False)

Importing **t20_wc2022_bowling_summary** json file

In [5]:
with open("/t20_wc2022_bowling_summary.json") as f:
  data = json.load(f)

  records = []
  for rec in data:
    records.extend(rec["bowlingSummary"])

df_bowling = pd.DataFrame(records)
df_bowling["match_id"] = df_bowling["match"].map(match_id_dict)
df_bowling.head(21)
df_bowling.to_csv("fact_bowling_summary.csv", index = False)

Importing **t20_wc2022_player_info** json file

In [6]:
with open("/t20_wc2022_player_info.json") as f:
  data = json.load(f)

df_player = pd.DataFrame(data)
df_player["name"] = df_player["name"].apply(lambda x : x.replace("†",""))
df_player.head()
df_player.to_csv("dim_players_summary.csv", index = False)